In [49]:
# Imports
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.prompts import  PromptTemplate
from langchain_core.documents import Document
from uuid import uuid4
from langchain_google_genai.chat_models import ChatGoogleGenerativeAI
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.chains.question_answering import load_qa_chain
import warnings
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
warnings.filterwarnings('ignore')
import chromadb
# Jupyter-specific imports
from IPython.display import display
# Set environment variable for protobuf
import os
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"
from dotenv import load_dotenv
import google.generativeai as genai


In [50]:
load_dotenv()
os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

In [51]:
# Load PDF
local_path = "AI.pdf"
if local_path:
    loader = UnstructuredPDFLoader(file_path=local_path)
    doc = loader.load()
    print(f"PDF loaded successfully: {local_path}")
else:
    print("Upload a PDF file")

PDF loaded successfully: AI.pdf


In [52]:
# Split text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=300)
chunks = text_splitter.split_documents(doc)
print(f"Text split into {len(chunks)} chunks")

Text split into 22 chunks


In [53]:
genai_embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [54]:

persistent_client = chromadb.PersistentClient(path="db/chroma_store")
vector_store_from_client = Chroma(
    client=persistent_client,
    collection_name="collection_name",
    embedding_function=genai_embedding,
)

In [55]:
documents = [Document(page_content=chunk.page_content, metadata=chunk.metadata) for chunk in chunks]
uuids = [str(uuid4()) for _ in range(len(documents))]
print("Documents:", documents)
print("UUIDs:", uuids)
print("Lengths match:", len(documents) == len(uuids))

Documents: [Document(metadata={'source': 'AI.pdf'}, page_content='BỘ GIÁO DỤC VÀ ĐÀO TẠO TRƯỜNG ĐẠI HỌC KINH TẾ QUỐC DÂN\n\nTIỂU LUẬN Học phần: Học máy (Machine Learning) Đề tài: Nghiên cứu và cài đặt chương trình phần mềm sử dụng kỹ thuật DecisionTreeCART để dự đoán chất lượng của các loại rượu. Đánh giá hiệu quả của mô hình phân lớp\n\nNhóm sinh viên Lớp học phần Giảng viên hướng dẫn : TS. Lưu Minh Tuấn\n\n: Nhóm 4 : 01\n\nHà Nội, Năm 2024\n\nMỤC LỤC CHƯƠNG 1: GIỚI THIỆU ĐỀ TÀI ......................................................................... 4\n\n1.1. Tính cấp thiết của đề tài ................................................................................ 4\n\n1.2. Mục tiêu của đề tài ........................................................................................... 4\n\n1.3. Đối tượng và phạm vi nghiên cứu ................................................................... 4\n\n1.4. Phương pháp nghiên cứu............................................................

In [56]:
try:
    vector_store_from_client.add_documents(documents = documents, ids=uuids)
    print("Documents added successfully!")
except Exception as e:
    print("Error:", e)

Documents added successfully!


In [57]:
genllm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-exp", temperature=0)

In [58]:

prompt_template = """
        You are an advanced AI assistant whose name is "Nexus" tasked with answering user {question} based on the provided document's extracted {context}
        You can also analyse data from CSV/Excel files reports, database.
        Use the following rules:
        - Always utilize all relevant context to generate a complete and accurate response.
        - If the context lacks information, clearly state: "Answer not found in the provided context."
        - Respond in the same language as the input question.
        - Provide structured responses (e.g., lists or bullet points) when applicable for clarity.
        - Prioritize the most relevant information and avoid irrelevant details.
        Context:
        {context}
        Question:
        {question}
        Response:.
        """
human_message = HumanMessagePromptTemplate.from_template(
    template=prompt_template
)


In [59]:
qa_prompt = ChatPromptTemplate.from_messages(
    [human_message]
)

In [60]:
# Set up retriever
retriever = vector_store_from_client.as_retriever(search_type="similarity",  search_kwargs={"k": 5})


In [61]:
qa_chain = load_qa_chain(genllm, chain_type="stuff", prompt=qa_prompt, verbose=True)


In [62]:
retriever_qa = RetrievalQA(combine_documents_chain=qa_chain, retriever=retriever, verbose=True)


In [65]:
def chat_with_pdf(question):
    """
    Chat with the PDF using the RAG chain.
    """
    return retriever_qa.invoke(question)

In [67]:
chat_with_pdf("Cam onon")



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Human: 
        You are an advanced AI assistant whose name is "Nexus" tasked with answering user Cam onon based on the provided document's extracted Chúng em xin chân thành cảm ơn!

3

CHƯƠNG 1: GIỚI THIỆU ĐỀ TÀI

1.1. Tính cấp thiết của đề tài

Chất lượng của rượu vang đóng vai trò quan trọng đối với cả người tiêu dùng lẫn ngành công nghiệp rượu. Phương pháp truyền thống để đánh giá chất lượng rượu thông qua các chuyên gia thường tốn nhiều thời gian và công sức. Hiện nay, các mô hình học máy đã trở thành công cụ quan trọng, thay thế một phần công việc của con người. Tuy nhiên, không phải tất cả các đặc điểm của rượu đều liên quan đến việc dự đoán chất lượng tốt hơn, mà chỉ có một số tính năng chính là cần thiết.

Khi đánh giá chất lượng của một chai rượu, mỗi người thường có quan điểm khác nhau. Đối với nhiều nhà phê bình rượu, chất lượng thư

{'query': 'Cam onon', 'result': 'Chúng em xin chân thành cảm ơn!\n'}